<a href="https://colab.research.google.com/github/IngaKristin/Final-Project-TensorFlow/blob/vera-branch/dataset_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Important Links:**
- [Paper](https://arxiv.org/pdf/2011.13062.pdf)
- [GitHub](https://github.com/sfc-computational-creativity-lab/x-rhythm-can)

1. [Donwload](https://magenta.tensorflow.org/datasets/groove#format) the Groove MIDI Dataset
2. Mount the dataset to Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import tensorflow as tfds